In [1]:
# !jupyter serverextension enable voila --sys-prefix
# !voila --enable_nbextensions=True

In [2]:
from ipywidgets import widgets, VBox
from PIL import ImageGrab
import pyautogui
import win32gui
import time
import numpy as np
from IPython.display import clear_output
import re
import requests
import lxml.html as lh
import asyncio
import pandas as pd
import webbrowser

# PokerStars Spin & Go Tracker

In [3]:
tables = {}

In [4]:
# df = pd.DataFrame(index=['Player 1', 'Player 2', 'Player 3'], columns=['Stack Size'])
# blinds = [10,20]

In [5]:
class HaltableExecutor:

    def __init__(self, togglebutton):
        self._togglebutton = togglebutton
        self.blinds = []
        self.stacks = []

    async def my_code(self):
        # This is your user code
        active_table_no = 0
        while True:
            get_tables()
            if self._togglebutton.value:
                drop_table.disabled = True
                btn_ranges.layout.visibility = 'visible'
                
                if active_table_no in tables:
                    active_table = tables[active_table_no]
                    lbl_table.value = active_table

                    self.blinds = list(map(int, re.search(r'\d+\/\d+', active_table).group().split('/')))
                    lbl_blinds.value = 'Blinds are <b>%d</b>/<b>%d</b>' %(self.blinds[0], self.blinds[1])
                    lbl_button.value = '<b>Player %d</b> is the button' % 1
                    
                    self.stacks = [150,350,450]
                    
                    lbl_player1.value = 'Player 1: <b>%d</b>' % self.stacks[0]
                    lbl_player2.value = 'Player 2: <b>%d</b>' % self.stacks[1]
                    lbl_player3.value = 'Player 3: <b>%d</b>' % self.stacks[2]
                    
                    for lbl in [lbl_blinds, lbl_button, lbl_player1, lbl_player2, lbl_player3]:
                        lbl.layout.visibility = 'visible'
                    
#                     out.clear_output(wait=True)                    
#                     with out: display(df)
                    
                else: btn_on_off.value = False
                
            else:
                drop_table.disabled = False
                
                for lbl in [lbl_blinds, lbl_button, lbl_player1, lbl_player2, lbl_player3, btn_ranges]:
                    lbl.layout.visibility = 'hidden'
                
#                 df.loc['Player 1'] = 0
#                 df.loc['Player 2'] = 0
#                 df.loc['Player 3'] = 0

#                 out.clear_output(wait=True)                
#                 with out: display(df)
                
                if len(tables) > 0:
                    active_table_no = list(tables.keys())[list(tables.values()).index(drop_table.value)]
                
            await asyncio.sleep(0.1)  # use this to temporarily give up control of the event loop to allow scheduling checkbox changes
        
    def start(self):
#         print('starting')
        asyncio.ensure_future(self.my_code())

In [6]:
def windowEnumerationHandler(hwnd, top_windows):
    top_windows.append((hwnd, win32gui.GetWindowText(hwnd)))


def get_tables():
    tables.clear()
    top_windows = []
    win32gui.EnumWindows(windowEnumerationHandler, top_windows)
    for i in top_windows:
        if "tournament" and "table" in i[1].lower():
            tables[i[0]] = i[1]
    
    if len(tables) > 0:
        drop_table.options = list(tables.values())
    
    else: drop_table.options = ['No open tables']
    
def get_table(table_id=None):
    tables.clear()
    top_windows = []
    win32gui.EnumWindows(windowEnumerationHandler, top_windows)
    for i in top_windows:
        if "tournament" and "table" in i[1].lower():
            tables[i[1]] = i[0]
        drop_table.options = list(tables.keys())
        
            
    if table_id == None:
        if len(tables) == 0:
            lbl_table.value = 'No tables!'
            return False
        else: 
            return tables
    
    else:
        for table in tables:
            if table[0] == table_id:
                return table[1]
        
        print('Table no longer available')
        return False

In [7]:
btn_on_off = widgets.ToggleButton(description='Tracking On/Off')

In [8]:
drop_table = widgets.Dropdown(options=[], 
                              value=None, # can use tuples of table_no, table_names here i fyou want
                              description='Select Table:',)

In [9]:
btn_ranges = widgets.Button(description='Get Ranges')

In [10]:
lbl_table = widgets.Label()
lbl_blinds = widgets.HTML()
lbl_player1 = widgets.HTML()
lbl_player2 = widgets.HTML()
lbl_player3 = widgets.HTML()
lbl_button = widgets.HTML()

In [11]:
# stacks = widgets.HTML(df.style.set_table_attributes('class="table"').render())

In [12]:
# def on_button_clicked(click):
#     get_tables()

In [13]:
def on_select(change):
    lbl_table.value = drop_table.value

In [14]:
def on_get_ranges(click):
    s1, s2, s3 = exe.stacks
    sb, bb = exe.blinds
    
    url = ('https://www.holdemresources.net/nashicm?action=calculate&s1=' + 
           str(s1) + '&s2=' + str(s2) + '&s3=' + str(s3) +
           '&p1=70&p2=30&p3=0&sb=' + str(sb) + '&bb=' + str(bb) + '&ante=0')
    webbrowser.open(url)

In [15]:
out = widgets.Output()

In [16]:
# btn_tables.on_click(on_button_clicked)
drop_table.observe(on_select, 'value')
btn_ranges.on_click(on_get_ranges)

In [17]:
get_tables()

In [18]:
display(VBox([btn_on_off, 
              widgets.HTML('<br>'), 
              drop_table, 
              widgets.HTML('<br>'), 
              lbl_table,
              lbl_blinds,
#               out,
              lbl_player1,
              lbl_player2,
              lbl_player3,
              lbl_button,
              btn_ranges
              ]))


In [19]:
exe = HaltableExecutor(btn_on_off)
exe.start()